In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import keras
import keras.layers as L
import math
from keras.utils import Sequence
from keras.preprocessing import image
from random import shuffle
from sklearn.model_selection import train_test_split
import plotly.express as px
import seaborn as sns
from sklearn.utils import class_weight

In [ ]:
from keras.models import Model,Sequential

In [ ]:
train_labels =pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
sample_submission = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')

In [ ]:
arr = np.load('../input/seti-breakthrough-listen/train/0/0000799a2b2c42d.npy').astype(np.float32)
fig = plt.figure(figsize=(16, 10))
for i in range(6):
    plt.subplot(6,1,i+1)
    plt.imshow(arr[i],aspect='auto')

In [ ]:
arr.shape

In [ ]:
arrnew = arr.reshape(6*273*256)

In [ ]:
arrnew.shape

In [ ]:
def on_target(arr):  #will return a new array that contain only on target array
    arr0 = arr[::2]
    return arr0

In [ ]:
fig = plt.figure(figsize=(16, 10))
arr_new = on_target(arr)
for i in range(3):
    plt.subplot(6,1,i+1)
    plt.imshow(arr_new[i],aspect='auto')

In [ ]:
def id_to_path(idx,train=True):
    path = '../input/seti-breakthrough-listen/'
    if train:
        folder = 'train/'
    else:
        folder = 'test/'
    path+=folder+idx[0]+'/'+idx+'.npy'
    return path

In [ ]:
class Dataset(Sequence):
    def __init__(self,idx,y=None,batch_size=8,shuffle=True):
        self.idx = idx
        self.batch_size = batch_size
        self.shuffle = shuffle
        if y is not None:
            self.is_train=True
        else:
            self.is_train=False
        self.y = y
    def __len__(self):
        return math.ceil(len(self.idx)/self.batch_size)
    def __getitem__(self,ids):
        batch_ids = self.idx[ids * self.batch_size:(ids + 1) * self.batch_size]
        if self.y is not None:
            batch_y = self.y[ids * self.batch_size: (ids + 1) * self.batch_size]
            
        list_x1 = np.array([np.load(id_to_path(x, self.is_train))[::2].reshape(3*273*256) for x in batch_ids])
        list_x3 = np.array([x.reshape(x.shape+(1,)) for x in list_x1])
        list_x2 = [list_x3 for i in range(3)]
        if self.is_train:
            return list_x2, batch_y
        else:
            return list_x2
    
    def on_epoch_end(self):
        if self.shuffle and self.is_train:
            ids_y = list(zip(self.idx, self.y))
            shuffle(ids_y)
            self.idx, self.y = list(zip(*ids_y))

In [ ]:
train_idx =  train_labels['id'].values
y = train_labels['target'].values
test_idx = sample_submission['id'].values

In [ ]:
x_train,x_valid,y_train,y_valid = train_test_split(train_idx,y,test_size=0.01,random_state=42)

In [ ]:
train_dataset = Dataset(x_train,y_train)
valid_dataset = Dataset(x_valid,y_valid)
test_dataset = Dataset(test_idx)

In [ ]:
x_valid.shape

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(train_labels['target'].values),train_labels['target'].values)
class_weights = dict(enumerate(class_weights))
class_weights

In [ ]:
from sklearn.model_selection import train_test_split 
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM,Dropout,concatenate
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Activation, Embedding, LSTM,Dropout,Bidirectional,GRU
from keras.utils import plot_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Flatten ,Embedding,Input,Conv1D,GlobalAveragePooling1D,GlobalMaxPooling1D,Dropout,MaxPooling1D,Bidirectional,GRU,Concatenate
from keras.models import Sequential,Model
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import keras

In [ ]:
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold, StratifiedKFold


In [ ]:
def crt_model():
    i1=Input(shape=(209664,1))
    l1=Conv1D(64,5,padding='valid', kernel_initializer='normal',activation='relu')(i1)
    l2 =MaxPooling1D(2) (l1)
    l3=Conv1D(128,5,padding='valid', kernel_initializer='normal',activation='relu')(l2)
    l3 =MaxPooling1D(2) (l3)
    l3=Conv1D(256,5,padding='valid', kernel_initializer='normal',activation='relu')(l3)
    l41=GlobalMaxPooling1D()(l3)
    
    
    
    i2=Input(shape=(209664,1))
    l12=Conv1D(64,5,padding='valid', kernel_initializer='normal',activation='relu')(i2)
    l22 =MaxPooling1D(2) (l12)
    l32=Conv1D(128,5,padding='valid', kernel_initializer='normal',activation='relu')(l22)
    l32 =MaxPooling1D(2) (l32)
    l32=Conv1D(256,5,padding='valid', kernel_initializer='normal',activation='relu')(l32)
    l42=GlobalMaxPooling1D()(l32)
    
    
    
    i3=Input(shape=(209664,1))
    l13=Conv1D(64,5,padding='valid', kernel_initializer='normal',activation='relu')(i3)
    l23 =MaxPooling1D(2) (l13)
    l33=Conv1D(128,5,padding='valid', kernel_initializer='normal',activation='relu')(l23)
    l33 =MaxPooling1D(2) (l33)
    l33=Conv1D(256,5,padding='valid', kernel_initializer='normal',activation='relu')(l33)
    l43=GlobalMaxPooling1D()(l33)
           
    
    l4 = concatenate([l41, l42, l43])
    
    
    l5=Dense(120, kernel_initializer='normal',activation='relu')(l4)
    l5=Dense(240, kernel_initializer='normal',activation='relu')(l5)
    l7=Dense(1, kernel_initializer='normal',activation='sigmoid')(l5)
    model=Model(inputs=[i1,i2,i3], outputs=l7)
    model.compile(optimizer='adam',loss='binary_crossentropy', metrics=[keras.metrics.AUC()])
    return model

In [ ]:
model=crt_model()

In [ ]:
history = model.fit(train_dataset,epochs=2,validation_data=valid_dataset,class_weight=class_weights)

In [ ]:
preds = model.predict(test_dataset)

In [ ]:
preds = preds.reshape(-1)

In [ ]:
submission = pd.DataFrame({'id':sample_submission['id'],'target':preds})
submission.to_csv('submission.csv',index=False)